#### 数据规格化

**数据准备**
+ 数据来源：University of Wisconsin的乳腺肿瘤数据，
+ 原始数据：
    - 原始属性数据来自于肿瘤细胞核图像的测量数据
    - 包括：半径，纹理，周长，面积，平滑性，紧致性，凹度，凹点数，对称性，分形维度
+ 衍生数据：
    - 由原始属性的均值、最大值、最小值等衍生出$30$维分类属性
+ 类别属性
    - Malignant：恶性
    - Benign：良性

In [21]:
import numpy as np
import pandas as pd

data = pd.read_csv("BreastCancer.csv")

class_mapping = {'M':0,'B':1}
data['diagnosis'] = data['diagnosis'].map(class_mapping)
data.head(16)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.26540,0.4601,0.11890,NaN
1,842517,0,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.18600,0.2750,0.08902,NaN
2,84300903,0,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.24300,0.3613,0.08758,NaN
3,84348301,0,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.25750,0.6638,0.17300,NaN
4,84358402,0,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.16250,0.2364,0.07678,NaN
5,843786,0,12.45,15.70,82.57,477.1,0.12780,0.17000,0.15780,0.08089,...,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.17410,0.3985,0.12440,NaN
6,844359,0,18.25,19.98,119.60,1040.0,0.09463,0.10900,0.11270,0.07400,...,27.66,153.20,1606.0,0.1442,0.2576,0.3784,0.19320,0.3063,0.08368,NaN
7,84458202,0,13.71,20.83,90.20,577.9,0.11890,0.16450,0.09366,0.05985,...,28.14,110.60,897.0,0.1654,0.3682,0.2678,0.15560,0.3196,0.11510,NaN
8,844981,0,13.00,21.82,87.50,519.8,0.12730,0.19320,0.18590,0.09353,...,30.73,106.20,739.3,0.1703,0.5401,0.5390,0.20600,0.4378,0.10720,NaN
9,84501001,0,12.46,24.04,83.97,475.9,0.11860,0.23960,0.22730,0.08543,...,40.68,97.65,711.4,0.1853,1.0580,1.1050,0.22100,0.4366,0.20750,NaN


**划分训练数据和测试数据**

In [17]:
from sklearn.model_selection import train_test_split

X = data.iloc[:,2:-1].to_numpy()
y = data.iloc[:,1].to_numpy()

print("Shape of X:", X.shape)
print("Number of Malignant:%d, Number of Benign:%d" %(y[y==0].size, y[y==1].size))

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

Shape of X: (569, 30)
Number of Malignant:212, Number of Benign:357


**使用未规格化数据学习和分类**

In [25]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(max_iter=1000,random_state=42)
mlp.fit(X_train, y_train)

print("Raw cancer data:")
print("\t Accuracy on training set: %4.3f" %(mlp.score(X_train, y_train)))
print("\t Accuracy on test set: %4.3f\n" %(mlp.score(X_test, y_test)))

Raw cancer data:
	 Accuracy on training set: 0.939
	 Accuracy on test set: 0.916



**使用规格化数据学习和分类**

+ 计算每一维数据的均值和方差：
$$ \mu = \frac{1}{n_{train}}\sum_{x\in X_{train}}{x},\quad \sigma^2 = \frac{1}{n_{train}}\sum_{x\in X_{train}}{(x-\mu)^2} $$
+ 规格化训练数据和测试数据，每一维属性单独归一化： 
$$ x \longleftarrow \frac{x-\mu}{\sigma} $$

In [26]:
mean_on_train = X_train.mean(axis=0)
std_on_train = X_train.std(axis=0)

X_train_scaled = (X_train - mean_on_train) / std_on_train
X_test_scaled = (X_test - mean_on_train) / std_on_train

mlp = MLPClassifier(max_iter=1000,random_state=0)
mlp.fit(X_train_scaled, y_train)

print("Normalized cancer data:")
print("\t Accuracy on training set: %4.3f" %(mlp.score(X_train_scaled, y_train)))
print("\t Accuracy on test set: %4.3f" %(mlp.score(X_test_scaled, y_test)))

Normalized cancer data:
	 Accuracy on training set: 1.000
	 Accuracy on test set: 0.972
